In [35]:
!pip show plotly

Name: plotly
Version: 2.0.14
Summary: Python plotting library for collaborative, interactive, publication-quality graphs.
Home-page: https://plot.ly/python/
Author: Chris P
Author-email: chris@plot.ly
License: MIT
Location: /anaconda/lib/python3.5/site-packages
Requires: nbformat, pytz, decorator, requests, six
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


<a id="load31"></a>
### 3.1 Get your Plotly credentials information
To create the visualization, you need to <a href="https://plot.ly/accounts/login/?action=signup" target="_blank">register with Plotly</a> then get the following keys for your Plotly account:
- API key  
- Streaming API key (create 2).

Paste that information in the cell below.

In [4]:
#Change the values in this cell with your username, api key, stream ids


plotly_api_key = 'yourkey'
plotly_stream_ids =['id0','id1']  # you need to create 2
plotly_username = 'yourid'



In [24]:
#Change the values in this cell with your username, api key, stream ids
import numpy as np
from plotly.graph_objs import *
import plotly
import plotly.plotly as pty
import plotly.tools 
import plotly.graph_objs as graph_objs

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from datetime import timedelta  
import datetime 

#Configure plotly to use your token
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key, stream_ids=plotly_stream_ids)

stream_tokens = plotly.tools.get_credentials_file()['stream_ids']
#In this case you just have the one token. If you wanted to graph multiple streams you would have to use multiple tokens.

user2_cfg = dict(token=plotly_stream_ids[0], maxpoints=10)
user1_cfg = dict(token=plotly_stream_ids[1], maxpoints=10)

start_time = datetime.datetime.fromtimestamp(1556152461)

user1_trace = go.Scatter(
    x=[start_time.strftime('%m-%d-%H:%M'), (start_time+timedelta(minutes=15)).strftime('%m-%d-%H:%M')],

    y=[0,0],
    mode='lines+markers',name="User1",
    stream=user1_cfg, marker = dict(
        size = 8,
        color = 'royalblue',
        line = dict(
            width = 1,
        )
    )         # (!) embed stream id, 1 per trace
)
# Initialize trace of streaming plot by embedding the unique stream_id
user2_trace = go.Scatter(
    x=[start_time.strftime('%m-%d-%H:%M'), (start_time+timedelta(minutes=15)).strftime('%m-%d-%H:%M')],
    y=[0,0],
    mode='lines+markers',
    name="User 2",
    stream=user2_cfg, marker = dict(
        size = 8,
        color = 'orange',
        line = dict(
            width = 2,
        )
    )         # (!) embed stream id, 1 per trace
)

data = go.Data([user2_trace, user1_trace])

# Add title to layout object
layout = go.Layout(title='User 1 vs User 2',   xaxis= dict(
        title= 'Time'
    ),
    yaxis=dict(
        title= 'Count'
    ),)

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# We will provide the stream link object the same token that's associated with the trace we wish to stream to
user2_stream = py.Stream(plotly_stream_ids[0])
user1_stream = py.Stream(plotly_stream_ids[1])
g_url = py.plot( fig,  filename='streaming-demo-3', auto_open=False, fileopt='overwrite')
tls.embed(g_url,width="85%",height=800)

In [30]:
def get_tsStr(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%m-%d-%H:%M')

sample_data = [{'user_2_val': 20.8, 'user_1_val': 26.0, 'user_2_time': '04-06-19:11', 'user_1_time': '04-06-18:41'},
{'user_2_val': 15.200000000000001, 'user_1_val': 14.0, 'user_2_time': '04-06-19:26', 'user_1_time': '04-06-18:56'},
{'user_2_val': 14.4, 'user_1_val': 6.0, 'user_2_time': '04-06-19:41', 'user_1_time': '04-06-19:11'},
{'user_2_val': 18.0, 'user_1_val': 18.0, 'user_2_time': '04-06-19:56', 'user_1_time': '04-06-19:26'},
{'user_2_val': 0, 'user_1_val': 4.0, 'user_2_time': '04-06-20:11', 'user_1_time': '04-06-19:41'}]

sample_data  = sorted(sample_data, key=lambda k: k['user_1_time']) 


In [34]:
import datetime

def get_tsStr(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%m-%d-%H:%M')

user2_stream.open()
user1_stream.open()


user1_stream.write(dict(x=[], y=[]))
user2_stream.write(dict(x=[], y=[]))

import time
time.sleep(2.0)
try:
    for reading in sample_data:
        user2_stream.write(dict(x=reading["user_2_time"] ,
                                     y=reading["user_2_val"]))
        user1_stream.write(dict(x = reading["user_1_time"], 
                                y = reading["user_1_val"]))
        time.sleep(1)
                   
except KeyboardInterrupt:
    pass

finally:
    # Leave group and commit final offsets
    user1_stream.close()
    user2_stream.close()
    print("Closed")

    
    
    

Closed
